# SPARTA (Semantic Parsing And Relational Table Aware)

This is a term project in `Unstructured Text Analysis` class.   
We implement the deep learning model for converting Korean language to SQL query. 

- github: https://github.com/TooTouch/SPARTA

<br>
<img src='https://user-images.githubusercontent.com/37654013/119700897-bec2c100-be8e-11eb-9d61-36de1ca66d5a.png'>
<br>

**Team Members**
- Hoonsang Yoon 
- Jaehyuk Heo 
- Jungwoo Choi
- Jeongseob Kim

**Information**
- Korea University [DSBA Lab](http://dsba.korea.ac.kr/)
- Advisor: [Pilsung Kang](http://dsba.korea.ac.kr/professor/)



In [1]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from konlpy.tag import Mecab

import os 
import sys
sys.path.append('../')

from train import construct_hyper_param, \
                  get_data, get_models, get_wemb_bert, \
                  sort_and_generate_pr_w, generate_sql_q, \
                  tokenize_corenlp_direct_version
from sqlova.utils.utils_wikisql import *
from sqlova.utils.utils import topk_multi_dim
from sqlnet.dbengine import DBEngine

import argparse
from IPython.display import display, clear_output
import ipywidgets

In [2]:
parser = argparse.ArgumentParser()
args = construct_hyper_param(parser, notebook=True)
device = 'cpu'

In [13]:
def inference(inputs, engine, datadir, bert_config, model_bert, tokenizer):
    nlu, nlu_t, hds, tb = inputs    
    
    # prediction
    wemb_n, wemb_h, l_n, l_hpu, l_hs, \
    nlu_tt, t_to_tt_idx, tt_to_t_idx = get_wemb_bert(bert_config, 
                                                     model_bert, 
                                                     tokenizer, 
                                                     nlu_t, 
                                                     hds, 
                                                     args.max_seq_length,
                                                     num_out_layers_n=args.num_target_layers, 
                                                     num_out_layers_h=args.num_target_layers,
                                                     device=device)

    prob_sca, prob_w, prob_wn_w, pr_sc, pr_sa, pr_wn, pr_sql_i, pr_sql_topk_i = model.beam_forward(wemb_n, 
                                                                                                   l_n, 
                                                                                                   wemb_h, 
                                                                                                   l_hpu,
                                                                                                   l_hs, 
                                                                                                   engine, 
                                                                                                   tb,
                                                                                                   nlu_t, 
                                                                                                   nlu_tt,
                                                                                                   tt_to_t_idx, 
                                                                                                   nlu,
                                                                                                   beam_size=args.beam_size,
                                                                                                   device=device)

    pr_wc, pr_wo, pr_wv, pr_sql_i = sort_and_generate_pr_w(pr_sql_i)

    pr_sql_q1 = generate_sql_q(pr_sql_i, tb)[0]
    pr_ans = engine.execute(tb[0]['id'], 
                            pr_sc[0], 
                            pr_sa[0], 
                            pr_sql_i[0]['conds'])
    
    return pr_sql_q1, pr_ans
    

In [14]:
args.datadir = '../data/ko_from_table'
args.logdir = '../logs/ko_from_table'
args.bert_name = 'bert-base-multilingual-cased'

In [15]:
_, _, _, _, test_data, test_table, _, _, test_loader = get_data(args.datadir, args)


# To start from the pre-trained models, un-comment following lines.
path_model_bert = os.path.join(args.logdir, 'model_bert_best.pt')
path_model = os.path.join(args.logdir, 'model_best.pt')
model, model_bert, tokenizer, bert_config = get_models(args, 
                                                       trained=True,
                                                       path_model_bert=path_model_bert, 
                                                       path_model=path_model, 
                                                       device=device)

In [16]:
engine = DBEngine(os.path.join(args.datadir, 'test.db'))

# Table information

In [39]:
def print_table_label(idx):
    
    tb = [test_table[test_data[idx]['table_id']]]
    hds = [tb[0]['header']]
    nlu_t = [test_data[idx]['question_tok']]
    nlu = [test_data[idx]['question']]
    sql = [test_data[idx]['sql']]

    # sql
    g_sql = generate_sql_q(sql, tb)[0]

    from_start_idx = g_sql.find('FROM')
    where_start_idx = g_sql.find('WHERE')
    if where_start_idx == -1:
        g_sql = g_sql[:from_start_idx] + '\n' + g_sql[from_start_idx:]
    else:
        g_sql = g_sql[:from_start_idx] + '\n' + g_sql[from_start_idx:where_start_idx] + '\n' + g_sql[where_start_idx:]

    # answer
    g_ans  = engine.execute(tb[0]['id'], 
                            sql[0]['sel'], 
                            sql[0]['agg'], 
                            sql[0]['conds'])[0]

    # plotting
    display(pd.DataFrame(tb[0]['rows'], columns=tb[0]['header']))

    print('='*20)
    print('Ground Truth')
    print('='*20)
    print('[Question]')
    print(nlu[0])
    print()
    print('[SQL]')
    print(g_sql)
    print()
    print('[RESULT]')
    print(g_ans)
    print('='*20)
    
    # inference
    display(ipywidgets.HTML('<h1>Inference</h1>'))
    question = ipywidgets.Text(description='QUESTION:', 
                           layout=ipywidgets.Layout(width='auto', display='flex'))
    button = ipywidgets.Button(description='TEST')
    output = ipywidgets.Output()
    display(ipywidgets.VBox([question, button]))
    display(output)
    
    def infer(b):
        q_tokenizer = Mecab()
        question_t = q_tokenizer.morphs(question.value)

        pr_sql_q1, pr_ans = inference(inputs=[[question.value], [question_t], hds, tb], 
                                      engine=engine,
                                      datadir=args.datadir, 
                                      bert_config=bert_config, 
                                      model_bert=model_bert, 
                                      tokenizer=tokenizer)
    
        # print results
        from_start_idx = pr_sql_q1.find('FROM')
        where_start_idx = pr_sql_q1.find('WHERE')
        if where_start_idx == -1:
            pr_sql_q1 = pr_sql_q1[:from_start_idx] + '\n' + pr_sql_q1[from_start_idx:]
        else:
            pr_sql_q1 = pr_sql_q1[:from_start_idx] + '\n' + pr_sql_q1[from_start_idx:where_start_idx] + '\n' + pr_sql_q1[where_start_idx:]
        
        with output:
            clear_output(wait=True)
            print('='*20)
            print('Inference')
            print('='*20)
            print('[SQL]')
            print(pr_sql_q1)
            print()
            print('[RESULT]')
            for ans in pr_ans:
                print(ans)
            print('='*20)
    
    button.on_click(infer)

In [37]:
index_text = ipywidgets.BoundedIntText(
    value=0,
    min=0,
    max=len(test_data),
    step=1,
    description='INDEX:',
    disabled=False
)

In [38]:
ipywidgets.interactive(print_table_label, idx=index_text)

interactive(children=(BoundedIntText(value=0, description='INDEX:', max=15878), Output()), _dom_classes=('widg…